In [1]:
# Install library for reading JPK File
%pip uninstall pyafmreader --yes
%pip install git+https://github.com/jlopezalo/pyafmreader.git

Found existing installation: pyafmreader x.x.x
Uninstalling pyafmreader-x.x.x:
  Successfully uninstalled pyafmreader-x.x.x
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/jlopezalo/pyafmreader.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-ggx3bx4a
  Running command git clone --filter=blob:none --quiet https://github.com/jlopezalo/pyafmreader.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-ggx3bx4a
  Resolved https://github.com/jlopezalo/pyafmreader.git to commit 455c83796458190d25d9ff9b6a4ad8b3e40680b4
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/TkTech/fasterzip.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-install-wm3m6r_1/fasterzip_5387e655b8cc4fe68b47d96e057adfd9
  Running command git clone --filter=blob:none --quiet https://github.com/TkTech/fasterzip.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-install-wm3m6r

In [2]:
# Install library for AFM data Rheology Analysis
%pip uninstall pyafmrheo --yes
%pip install git+https://github.com/jlopezalo/pyafmrheo.git

Found existing installation: pyafmrheo 0.0.1
Uninstalling pyafmrheo-0.0.1:
  Successfully uninstalled pyafmrheo-0.0.1
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/jlopezalo/pyafmrheo.git to /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-31ydoe01
  Running command git clone --filter=blob:none --quiet https://github.com/jlopezalo/pyafmrheo.git /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-req-build-31ydoe01
  Resolved https://github.com/jlopezalo/pyafmrheo.git to commit 43295b580a50f00097694a503bc5f35d11e24c98
  Preparing metadata (setup.py) ... done
  Created wheel for pyafmrheo: filename=pyafmrheo-0.0.1-py3-none-any.whl size=33274 sha256=3754c6ee9a2c95e727b9b4b8868712d487fd1221d3d988b8fd2dd6a130353fdb
  Stored in directory: /private/var/folders/4k/ps22fmxj3gx6vls78t2p63wh0000gn/T/pip-ephem-wheel-cache-21myl1jl/wheels/45/a3/5e/2d19803e3cbf5fe45f904090127b31ec30fc304c6b6e5e0f70
Successfully built pya

In [3]:
# Import libraries we will need
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Get file reader from library
from pyafmreader import loadfile

# Get data analysis tools
from pyafmrheo.routines.HertzFit import doHertzFit

In [4]:
# Define global variables
file_path = '/Users/javierlopez/Documents/Datasets/05-11-2021/E/map-data-2021.11.05-15.41.45.699.jpk-force-map'
# Shapes available: paraboloid, pyramid
indenter_shape = "pyramid"
# tip_parameter = 500 * 1e-9 # meters
tip_parameter = 35 # degrees
# Poisson ratio
poisson_ratio = 0.5
# Window to find cp
windowforCP = 350e-9
# If None it will use the deflection sensitivity from the file
deflection_sensitivity = None # m/V
# If None it will use the spring constant from the file
spring_constant = None # N/m
# General plotting params
plt.rcParams["figure.figsize"] = (10,5)

In [5]:
file = loadfile(file_path)

In [6]:
filemetadata = file.filemetadata
print(filemetadata['file_type'])

.jpk-force-map


In [7]:
closed_loop = filemetadata['z_closed_loop']
file_deflection_sensitivity = filemetadata['defl_sens_nmbyV'] #nm/V
file_spring_constant = filemetadata['spring_const_Nbym'] #N/m
height_channel = filemetadata['height_channel_key']

if not deflection_sensitivity: deflection_sensitivity = file_deflection_sensitivity / 1e9 #m/V
if not spring_constant: spring_constant = file_spring_constant

In [8]:
param_dict = {
    'def_sens': deflection_sensitivity,
    'k': spring_constant,
    'height_channel': height_channel,
    'poc_win': windowforCP,
    'curve_seg': 'extend',
    'fit_range_type': 'indentation',
    'min_ind': -np.inf,
    'max_ind': np.inf,
    'fit_line': False,
    'd0': 0.0,
    'f0': 0.0,
    'auto_init_E0': True,
    'correct_tilt': True,
    'tilt_max_offset': 1e-6,
    'tilt_min_offset': 10e-9,
    'contact_model': indenter_shape,
    'tip_param': tip_parameter
}

In [9]:
curve_idx = 0
force_curve = file.getcurve(curve_idx)
force_curve.preprocess_force_curve(param_dict['def_sens'], param_dict['height_channel'])
if filemetadata['file_type'] in ('.jpk-force', '.jpk-force-map', '.jpk-qi-data'):
    force_curve.shift_height()

In [10]:
hertz_result = doHertzFit(force_curve, param_dict)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pyafmrheo/models/hertz.py:161: RuntimeWarning: divide by zero encountered in true_divide
  a = (self.get_residuals(indentation, force, sample_height)**2/force)


In [11]:
hertz_result.fit_report()


        # Fit parameters
        Indenter shape: pyramid

        Tip paraneter: 35

        BEC Model: None

        Number of free parameters: 3

        delta0: -2.92122260780748e-07

        E0: 345.270063369498

        f0: -4.143997372771431e-12

        slope: None

        # Fit metrics
        MAE: -8.61250023318415e-19

        MSE: 1.9267237439451726e-22

        RMSE: 1.3880647477496043e-11

        Rsq: 0.9911992087067769

        Chisq: 2.0852635473822456e-08

        RedChisq: 6.950878491274152e-09

        
